In [232]:
import math
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [233]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

This model approximates the function f(x) = 1/1+e^-x

In [234]:
def int_to_64bit_list(number):
    binary_str = format(number, '064b')
    return [int(bit) for bit in binary_str]

def float_to_64bit_list(number):
    binary_str = format(int(number * 2**63), '064b')
    return [int(bit) for bit in binary_str]

In [235]:
# test dataset
logistic = lambda x: 1 / (1 + math.exp(-x))

ints = [x for x in range(100_000)]

input = [int_to_64bit_list(x) for x in ints]

labels = [float_to_64bit_list(logistic(x)) for x in ints]

print(len(input[37]))

64


In [236]:
t = torch.Tensor(input)
t[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [237]:
def recover(x):
  binary_str = ''.join(str(bit) for bit in x)
  integer_value = int(binary_str, 2)
  return integer_value / 2**64

In [238]:
# test converting floats to 64-bit binary and back

test_set = [0.75, 0.5, 0.25, 0.125, 0.0625, 0.03125]

for x in test_set:
    print(f"{x} -> {recover(float_to_64bit_list(x))}")

0.75 -> 0.375
0.5 -> 0.25
0.25 -> 0.125
0.125 -> 0.0625
0.0625 -> 0.03125
0.03125 -> 0.015625


In [239]:
class Data(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.labels = outputs

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)

In [240]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()  
        self.layer1 = nn.Linear(64, 512 * 5) # 64 bits in
        self.layer2 = nn.Linear(512 * 5, 512 * 10)  
        self.output = nn.Linear(512 * 10, 64) # 64 bits out

    def forward(self, x):
        # try relu to begin with
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.output(x)
        return x

In [241]:
training_dataset = Data(input, labels)
training_loader = torch.utils.data.DataLoader(training_dataset,batch_size=128)

model = Model().to(device)

loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(10):
    for data in training_loader:
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
        loss_size = loss(outputs, labels.to(device))
        loss_size.backward()
        optimizer.step()

    print(f"Loss: {loss_size.item()}")

torch.save(model.state_dict(), "model")

Loss: -0.0
Loss: 2.864741190933273e-06
Loss: -0.0
Loss: -0.0
Loss: -0.0
Loss: -0.0
Loss: -0.0
Loss: -0.0
Loss: -0.0
Loss: -0.0
